In [52]:
# Imports
import plotly.express as px
import panel as pn
import pandas as pd
import os
from pathlib import Path
from dotenv import load_dotenv
import yfinance as yf
import requests
import hvplot.pandas
import json


In [53]:
# Setup all DataFrames
# Gold

gold = yf.download('GC=F', period='max')
gold_data = pd.DataFrame(gold['Close']).pct_change().dropna().rename(columns={"Close": "Gold"})

# Oil

oil = yf.download("CL=F", period='max')
oil_data = pd.DataFrame(oil['Close']).pct_change().dropna().rename(columns={"Close": "Oil"})

# S&P 500

snp = yf.download('^GSPC', period='max')
snp_data = pd.DataFrame(snp['Close']).pct_change().dropna().rename(columns={"Close": "S&P 500"})

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [54]:
today = pd.to_datetime('today')
print(today)

2022-04-04 21:08:01.355735


In [55]:
# Slice out Data
# GFC

gold_data_gfc = gold_data.loc["2006-06-01":"2010-01-01"]
oil_data_gfc = oil_data.loc["2006-06-01":"2010-01-01"]
snp_data_gfc = snp_data.loc["2006-06-01":"2010-01-01"]

# 9/11 and Afghan

gold_data_911 = gold_data.loc["2000-10-07":"2005-08-30"]
oil_data_911 = oil_data.loc["2000-10-07":"2005-08-30"]
snp_data_911 = snp_data.loc["2000-10-07":"2005-08-30"]

# COVID-19

gold_data_covid = gold_data.loc["2018-12-31":today]
oil_data_covid = oil_data.loc["2018-12-31":today]
snp_data_covid = snp_data.loc["2018-12-31":today]

# Potential long-term effects of war from the analysis of previous war data? Ukraine Russia War

gold_data_long = gold_data.loc["2000-10-07":"2012-12-15"]
oil_data_long = oil_data.loc["2000-10-07":"2012-12-15"]
snp_data_long = snp_data.loc["2000-10-07":"2012-12-15"]

# What are the current short-term effects of war? (Ukraine Russia War)

gold_data_short = gold_data.loc["2022-02-24":today]
oil_data_short = oil_data.loc["2022-02-24":today]
snp_data_short = snp_data.loc["2022-02-24":today]


In [56]:
# Concat by every event
# GFC
event_gfc = pd.concat(
    [gold_data_gfc, oil_data_gfc, snp_data_gfc], axis="columns", join="inner"
)
# 9/11

event_911 = pd.concat(
    [gold_data_911, oil_data_911, snp_data_911], axis="columns", join="inner"
)

# COVID-19

event_covid = pd.concat(
    [gold_data_covid, oil_data_covid, snp_data_covid], axis="columns", join="inner"
)

# Potential long-term effects of war from the analysis of previous war data? Ukraine Russia War

event_long = pd.concat(
    [gold_data_long, oil_data_long, snp_data_long], axis="columns", join="inner"
)

# What are the current short-term effects of war? (Ukraine Russia War)

event_short = pd.concat(
    [gold_data_short, oil_data_short, snp_data_short], axis="columns", join="inner"
)